<a href="https://colab.research.google.com/github/bill7845/colab_repo/blob/master/dacon_%EC%A0%9C%EC%A3%BC%EC%B9%B4%EB%93%9C%EC%82%AC%EC%9A%A9%EB%9F%89%EC%98%88%EC%B8%A1/modeling_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/201901-202003.csv', encoding='utf-8')
data.shape

(24697792, 12)

In [3]:
data.head(1)

,REG_YYMM,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT
0,201901,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4


In [4]:
## data preprocessing

data = data.fillna('세종') # 결측값 ''로 대체

## 날짜 처리
def grap_year(data): # year 
    data = str(data)
    return int(data[:4])

def grap_month(data): # month 
    data = str(data)
    return int(data[4:])

data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1) # 기존 날짜컬럼 삭제
data.head()

,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,HOM_SIDO_NM,HOM_CCG_NM,AGE,SEX_CTGO_CD,FLC,CSTMR_CNT,AMT,CNT,year,month
0,강원,강릉시,건강보조식품 소매업,강원,강릉시,20s,1,1,4,311200,4,2019,1
1,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,1,2,7,1374500,8,2019,1
2,강원,강릉시,건강보조식품 소매업,강원,강릉시,30s,2,2,6,818700,6,2019,1
3,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,3,4,1717000,5,2019,1
4,강원,강릉시,건강보조식품 소매업,강원,강릉시,40s,1,4,3,1047300,3,2019,1


In [5]:
# 결측치 확인
data.isnull().sum()

CARD_SIDO_NM    0
CARD_CCG_NM     0
STD_CLSS_NM     0
HOM_SIDO_NM     0
HOM_CCG_NM      0
AGE             0
SEX_CTGO_CD     0
FLC             0
CSTMR_CNT       0
AMT             0
CNT             0
year            0
month           0
dtype: int64

In [6]:
# 시군구도 필요없으니 시군구 변수도 삭제
del data['CARD_CCG_NM']
del data['HOM_CCG_NM']
del data['CSTMR_CNT']
del data['CNT']

In [7]:
sex_condi = {1:0, 2:1} # 남자 0 여자 1로 인코딩
data['SEX_CTGO_CD'] = data['SEX_CTGO_CD'].map(sex_condi)

# 현지인, 외지인 변수 추가
def get_local_nlocal(df):
  if df['CARD_SIDO_NM'] == df['HOM_SIDO_NM']:
    return 1 # 현지인 1
  else :
    return 0 # 외지인 0

# data['local_nlocal'] = data.apply(get_local_nlocal,axis=1)

data = data[['year','month','CARD_SIDO_NM','STD_CLSS_NM','HOM_SIDO_NM','AGE','SEX_CTGO_CD','FLC','AMT']] # 'local_nlocal',

In [8]:
data.head()

,year,month,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT
0,2019,1,강원,건강보조식품 소매업,강원,20s,0,1,311200
1,2019,1,강원,건강보조식품 소매업,강원,30s,0,2,1374500
2,2019,1,강원,건강보조식품 소매업,강원,30s,1,2,818700
3,2019,1,강원,건강보조식품 소매업,강원,40s,0,3,1717000
4,2019,1,강원,건강보조식품 소매업,강원,40s,0,4,1047300


* 지역별 예측

In [9]:
data

,year,month,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT
0,2019,1,강원,건강보조식품 소매업,강원,20s,0,1,311200
1,2019,1,강원,건강보조식품 소매업,강원,30s,0,2,1374500
2,2019,1,강원,건강보조식품 소매업,강원,30s,1,2,818700
3,2019,1,강원,건강보조식품 소매업,강원,40s,0,3,1717000
4,2019,1,강원,건강보조식품 소매업,강원,40s,0,4,1047300
...,...,...,...,...,...,...,...,...,...
24697787,2020,3,충북,휴양콘도 운영업,충북,30s,0,2,43300
24697788,2020,3,충북,휴양콘도 운영업,충북,40s,0,3,35000
24697789,2020,3,충북,휴양콘도 운영업,충북,50s,0,4,188000
24697790,2020,3,충북,휴양콘도 운영업,충북,50s,1,4,99000


In [13]:
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from itertools import product

# encoding def
def encoder_func(df):
  dtypes = df.dtypes
  encoders = {}
  for column in df.columns:
    if str(dtypes[column]) == 'object':
      encoder = LabelEncoder()
      encoder.fit(df[column])
      encoders[column] = encoder

  for column in encoders.keys():
    encoder = encoders[column]
    df[column] = encoder.transform(df[column])

  return df,encoders

submission = pd.read_csv('/content/drive/My Drive/Colab code/dacon_카드사용액 예측/data/submission.csv', index_col=0)

# 지역별 모델링
def modeling(full_data,submission):

  res_model_dict = {}
  res_forecasting = pd.DataFrame(columns=['REG_YYMM','CARD_SIDO_NM','STD_CLSS_NM','AMT'])

  city_list = full_data['CARD_SIDO_NM'].unique()
  
  for city in city_list:
    sampling = full_data.query("CARD_SIDO_NM == @city")
    sampling,encoders = encoder_func(sampling)

    train_num = sampling.sample(frac=1, random_state=0)
    train_features = train_num.drop(['CARD_SIDO_NM','AMT'],axis=1)
    train_target = np.log1p(train_num['AMT'])

    model = RandomForestRegressor(n_jobs=-1, random_state=0)
    model.fit(train_features, train_target)

    res_model_dict['model_'+city] = model

    year_temp = [2020]
    month_temp = [4,7]
    STD_CLSS_NM_temp = sampling['STD_CLSS_NM'].unique()
    HOM_SIDO_NM_temp = sampling['HOM_SIDO_NM'].unique()
    AGE_temp = sampling['AGE'].unique()
    SEX_CTGO_CD_temp = sampling['SEX_CTGO_CD'].unique()
    FLC_temp = sampling['FLC'].unique()
    # local_nlocal_temp = sampling['local_nlocal'].unique()
    comb_list = [year_temp, month_temp, STD_CLSS_NM_temp, HOM_SIDO_NM_temp, AGE_temp, SEX_CTGO_CD_temp, FLC_temp] # ,'local_nlocal_temp'

    forecasting_temp = np.array(list(product(*comb_list)))
    forecasting_temp = pd.DataFrame(data=forecasting_temp, columns=train_features.columns)

    pred = model.predict(forecasting_temp)
    pred = np.expm1(pred)
    forecasting_temp['AMT'] = np.round(pred, 0)
    forecasting_temp['REG_YYMM'] = forecasting_temp['year']*100 + forecasting_temp['month']
    forecasting_temp = forecasting_temp[['REG_YYMM','STD_CLSS_NM', 'AMT']]
    forecasting_temp.insert(1,'CARD_SIDO_NM',city)
    forecasting_temp = forecasting_temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

    forecasting_temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(forecasting_temp['STD_CLSS_NM'])
    
    res_forecasting = pd.concat([res_forecasting,forecasting_temp])
    

  return res_model_dict,res_forecasting

res_model_dict,res_forecasting = modeling(data,submission)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [ ]:
res_model_dict

In [14]:
res_forecasting

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,243636508.0
1,202004,강원,골프장 운영업,456914849.0
2,202004,강원,과실 및 채소 소매업,154622977.0
3,202004,강원,관광 민예품 및 선물용품 소매업,72058626.0
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,87215050.0
...,...,...,...,...
73,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,60615882.0
74,202007,충북,한식 음식점업,172567551.0
75,202007,충북,호텔업,119980303.0
76,202007,충북,화장품 및 방향제 소매업,149281441.0


In [15]:
submission

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,0
1,202004,강원,골프장 운영업,0
2,202004,강원,과실 및 채소 소매업,0
3,202004,강원,관광 민예품 및 선물용품 소매업,0
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,0
...,...,...,...,...
1389,202007,충북,피자 햄버거 샌드위치 및 유사 음식점업,0
1390,202007,충북,한식 음식점업,0
1391,202007,충북,호텔업,0
